# Tourism

This example highlights how you can actually use a combination of deliberately extracting blank cells along with horizontal conditions to unpick complex (bad) table structures.

## Tutorial Structure

With these example tutorials I'm going to comment heavily and cover nuances in a follow up section (with liberal targetted previews as needed) as it's the easiest way to grapple with new ideas. It may also be worth opening up these notebooks yourself (they're in `./jupyterbook` in the [tidychef](https://github.com/mikeAdamss/tidychef) github repo) so you can run, alter and generally have a play about with this yourself as part of your learning.

We'll cover:

- source data
- requirements, what we're aiming to do here
- show the full script (all logic commented)
- output the selection preview
- nuances (where applicable)
- view the output

This sequencing is necessary as the output for some of the example is **really** long so that necessitates it coming last. If you're viewing this via a jupyter book (i.e on the site) you can navigate between the above sections via your right hand menu.

_Note - these tutorial scripts might seem verbose due all the comments but that's ok (this is a tutorial after all). If you take them out you end up with a fairly succinct and human readable encapsulation of what would otherwise (with existing tools) be a rather convoluted and fragile set of instructions to express._

In virtually all cases I'll make heavy use of `preview` and `bounded` to only look at relevant parts of what can be quite large datasets. Downloads links are provided for the source data.

## Source

For this example we're extracting the table3 from an xlsx dataset dealing with travel and tourism.

In [ ]:
from tidychef import acquire, preview
from tidychef.selection import XlsxSelectable

table: XlsxSelectable = acquire.xlsx.http("https://raw.githubusercontent.com/mikeAdamss/tidychef/main/tests/fixtures/xlsx/tourism.xlsx", tables="Table 3")
preview(table, bounded="A1:L20")

## Requirements

- Rows 4 and 5 combines as "Country"
- Column A as "Year"
- Column B as "Month".
- Row 6 as "Seasonal Adjustment"

We're not gonna take the footer data dealing with three month comparissons for our purposes here.


In [ ]:
from typing import Dict, List
from tidychef import acquire, preview
from tidychef.direction import up, down, left, right
from tidychef.output import Column, TidyData
from tidychef.selection import XlsxSelectable

tables: List[XlsxSelectable] = acquire.xlsx.http("https://raw.githubusercontent.com/mikeAdamss/tidychef/main/tests/fixtures/xlsx/tourism.xlsx")
table = tables[3]

# We'll fine seasonal adjustment by looking for the row containing both SA and NSA
seasonal_adjustment = table.row_containing_strings(["SA", "NSA"]).is_not_blank().label_as("Seasonal Adjustment")

# With area, we're going to take all three rows and concatenate them later with a horizontal condition
area1 = seasonal_adjustment.shift(up(3)).label_as("area1")
area2 = seasonal_adjustment.shift(up(2)).label_as("area2")
area3 = seasonal_adjustment.shift(up(1)).label_as("area3")

# Month is things in column B (not row 1) thats not blank
month = table.excel_ref("B2").fill(down).is_not_blank().label_as("Month")

# Year is one cell to the left of month
year = month.shift(left).is_not_blank().label_as("Year")

# Observations are at the intersection of month and seasonal adjustment so we'll just use a waffle.
observations = month.waffle(right, seasonal_adjustment).label_as("Observations")

preview(seasonal_adjustment, area1, area2, area3, month, year, observations, bounded="A1:L20")

# This is the function for the horizontal condition, we'll discuss this in nuances
def create_area(line) -> str:
    """
    A function to create a sensible string from the three areas
    """
    # We need to get rid of any trailing whitespace and remove the non header text mised in with the headers
    area1 = line["area1"].replace("Area of visit", "").strip()
    area2 = line["area2"].replace("of which", "").replace("Other countries", "").strip()
    area3 = line["area3"].strip()
    
    # Combined the header
    # If its blank its column B, so set to all visits
    combined_area = f"{area1} {area2} {area3}"
    return combined_area if combined_area.strip() != "" else "All Visits"
    
tidy_data = TidyData(
    observations,
    Column(seasonal_adjustment.attach_directly(down)),
    Column(area1.attach_directly(down)),
    Column(area2.attach_directly(down)),
    Column(area3.attach_directly(down)),
    Column(month.attach_directly(right)),
    Column(year.attach_closest(down)),
    Column.horizontal_condition("Area", create_area), # this is us using the create_area function to populate a column
    drop=["area1", "area2", "area3"]
)

## Nuances

### The Horizontal Condition + Drop

At this point it'd be worth having a read of the documentation for Horizontal Conditions (it's not long). The following will recap that a little, but we are assuming you have read it.

So a column made by `Column.horizontal_condition(<column name>, <callable>)` just uses passes a dictionary **representing the rest of the row** to the callable.

To make this clearer lets look at the first line of what the tidy data would look like **without** dropping any columns.


In [ ]:
temporary_tidy_data = TidyData(
    observations,
    Column(seasonal_adjustment.attach_directly(down)),
    Column(area1.attach_directly(down)),
    Column(area2.attach_directly(down)),
    Column(area3.attach_directly(down)),
    Column(month.attach_directly(right)),
    Column(year.attach_closest(down)),
    Column.horizontal_condition("Area", create_area),
    # drop=["area1", "area2", "area3"]   - this is me not dropping the temporary columns
)

# Note: tidychef doesnt transform until the tidy data is needed (usually when you to_csv) so we'll need to prod it along and truncate the outut
# so I can demonstrate what we're working with.
# This is more "under the hood" trickery than intended usage (hence _ prefixes) but we need a quick example.
# i.e don't worry about how I'm doing this, there's no reason for you to
temporary_tidy_data._transform()
temporary_tidy_data._data = temporary_tidy_data._data[:2] 
print("These are the column headers and the first row\n", temporary_tidy_data)

When you use `horizontal_condition` if has access to the contents of a given row as a dictionary, so the example row would be:

```json
{
    "Observations": "5831.281022512688",
    "Seasonal Adjustment": "NSA",
    "area1": "All Visits",
    "area2": "",
    "area3": "",
    "Month": "Jan",
    "Year": "2018",
}
```

which hopefully conextualises what this function is doing. tThe `line` variable is the contents of _a_ row and populates the `Area` column for _that row_. i.e in the case of our **example first row** `line` would be the contents of the above dictionary.

```python
# This is the function for the horizontal condition, we'll discuss in nuances
def create_area(line) -> str:
    """
    A function to create a sensible string from the three areas
    """
    # Area 2 needs a little extra handling as we don't want "of which"
    area1 = line["area1"].replace("Area of visit", "").strip()
    area2 = line["area2"].replace("of which", "").replace("Other countries", "").strip()
    area3 = line["area3"].strip()
    
    # Combined the header
    # If its blank its column B, soset to all visits
    combined_area = f"{area1} {area2} {area3}"
    return combined_area if combined_area.strip() != "" else "All Visits"
```

Which for our **single example row** returns the "All Visits" value which populates the `Area` column (the horizontal condition column).

All the other rows are calcualted in the same way - the line dictionary goes to the function - the function populate `Area`.

The `drop` keyword just gets rid of the intermediary columns at the end (when we've no use for them).

# Outputs

The tidy data can be [downloaded here](./tourism.csv) and a full inline preview of the tidydata generated is shown below for those people who'd prefer to scroll.

In [ ]:
print(tidy_data)